In [57]:
import os

os.environ['KERAS_BACKEND'] = "torch"

In [58]:
import pandas as pd

text_dataset: list[str] = pd.read_csv("../output/dataset/pre-processed/100k.preprocessed.csv")["comments"]

In [59]:
from gensim import corpora

# Make Dictionary:
tokenized_text = [doc.split(" ") for doc in text_dataset]
dictionary = corpora.Dictionary(tokenized_text)

corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in tokenized_text]

In [60]:
from gensim.models.ldamodel import LdaModel

random_state = 1234
# todo scout for best topic_size
model = LdaModel(
    corpus=corpus, id2word=dictionary, num_topics=14,
    random_state=random_state, chunksize=64, per_word_topics=True, passes=10
)

In [61]:
from pprint import pprint

pprint(model.print_topics(num_words=20))
doc_lda = model[corpus]

[(0,
  '0.133*"play" + 0.113*"game" + 0.044*"<date>" + 0.040*"tile" + 0.037*">" + '
  '0.036*"look" + 0.033*"expansion" + 0.031*"nice" + 0.027*"solo" + '
  '0.026*"similar" + 0.025*"draw" + 0.020*"component" + 0.017*"review" + '
  '0.015*"allow" + 0.015*"interaction" + 0.014*"art" + 0.013*"hour" + '
  '0.013*"artwork" + 0.012*"count" + 0.012*"family"'),
 (1,
  '0.120*"like" + 0.111*"game" + 0.084*"feel" + 0.037*"interesting" + '
  '0.034*"play" + 0.034*"use" + 0.028*"worker" + 0.024*"big" + 0.024*"area" + '
  '0.021*"mix" + 0.019*"placement" + 0.018*"small" + 0.018*"power" + '
  '0.017*"control" + 0.016*"sure" + 0.015*"random" + 0.015*"event" + '
  '0.013*"variety" + 0.013*"thematic" + 0.012*"kind"'),
 (2,
  '0.136*"need" + 0.063*"design" + 0.050*"level" + 0.048*"box" + 0.040*"age" + '
  '0.029*"edition" + 0.029*"etc" + 0.023*"leave" + 0.023*"sell" + 0.023*"say" '
  '+ 0.021*"world" + 0.020*"core" + 0.019*"kid" + 0.018*"face" + 0.018*"stuff" '
  '+ 0.016*"open" + 0.015*"wait" + 0.015*"

In [62]:
from gensim.models import CoherenceModel

# Evaluation
topn = 10
coh_model = CoherenceModel(model, corpus=corpus, coherence='u_mass', topn=10)

In [63]:
coh_model.get_coherence_per_topic()

[-4.404440461701165,
 -3.6558205865598152,
 -4.689687644482565,
 -3.5379252056109163,
 -2.7824415954970725,
 -3.786175727998033,
 -3.5313237395888057,
 -4.977956936438127,
 -4.27561679633778,
 -3.0251431670106266,
 -3.9700140556842736,
 -3.0500927099569752,
 -4.107494263397601,
 -4.227697966893872]

In [56]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18    -0.111426 -0.348924       1        1  8.726660
9     -0.239049 -0.134582       2        1  7.540632
15     0.044471  0.037044       3        1  7.187532
10    -0.138012 -0.045361       4        1  7.075791
13    -0.101455  0.091352       5        1  7.015102
3     -0.052762  0.099573       6        1  5.369268
1     -0.014887  0.072305       7        1  5.348236
0     -0.204552 -0.102241       8        1  4.894779
16    -0.038942  0.080579       9        1  4.785822
11    -0.053416  0.083344      10        1  4.642418
8      0.217562 -0.077708      11        1  4.603397
14    -0.005045  0.109509      12        1  4.565317
4     -0.042711  0.113597      13        1  4.187582
12     0.225895 -0.096551      14        1  4.133812
7     -0.021633  0.077788      15        1  4.077436
19    -0.016321  0.076803      16        1  3.765627
17     0.224339 -0.072389      17        1  3.401793
2      0.153850 -0.005484      18        1  3.209400
6     -0.044839  0.081870      19        1  2.839565
5      0.218933 -0.040525      20        1  2.629832, topic_info=             Term          Freq         Total Category  logprob  loglift
30          <num>  67190.000000  67190.000000  Default  30.0000  30.0000
0     <game_name>  39240.000000  39240.000000  Default  29.0000  29.0000
99           card  34182.000000  34182.000000  Default  28.0000  28.0000
62           play  49535.000000  49535.000000  Default  27.0000  27.0000
48         player  32129.000000  32129.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
914          take      0.102957      1.753201  Topic20 -12.7427   0.8034
1002        right      0.102946      1.752847  Topic20 -12.7428   0.8035
910          have      0.102943      1.752727  Topic20 -12.7428   0.8035
878         table      0.102940      1.752626  Topic20 -12.7428   0.8035
858          give      0.102943      1.752869  Topic20 -12.7428   0.8034

[746 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
123       9  0.999748            +
205       1  0.999735       <date>
0         6  0.999955  <game_name>
30        1  0.999970        <num>
416       4  0.998986            >
...     ...       ...          ...
59       15  0.999668         work
534       9  0.999707       worker
215      18  0.999221        world
481      16  0.999038        worth
491       5  0.911192            z

[425 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 10, 16, 11, 14, 4, 2, 1, 17, 12, 9, 15, 5, 13, 8, 20, 18, 3, 7, 6])